## **[Optimizing Pandas Code: The Impact of Operation Sequence](https://towardsdatascience.com/optimizing-pandas-code-the-impact-of-operation-sequence-0c5aa159632a)**

In [10]:
## 사전형데이터 축약문
{
    letter: list(range(10))
    for letter in "abcde"
}

{'a': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'b': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'c': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'd': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'e': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [1]:
import pandas as pd

n = 1_000_000
df = pd.DataFrame({
    letter: list(range(n))
    for letter in "abcdefghijklmnopqrstuwxyz"
})

In [2]:
df

,a,b,c,d,e,f,g,h,i,j,...,p,q,r,s,t,u,w,x,y,z
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,999995,999995,999995,999995,999995,999995,999995,999995,999995,999995,...,999995,999995,999995,999995,999995,999995,999995,999995,999995,999995
999996,999996,999996,999996,999996,999996,999996,999996,999996,999996,999996,...,999996,999996,999996,999996,999996,999996,999996,999996,999996,999996
999997,999997,999997,999997,999997,999997,999997,999997,999997,999997,999997,...,999997,999997,999997,999997,999997,999997,999997,999997,999997,999997
999998,999998,999998,999998,999998,999998,999998,999998,999998,999998,999998,...,999998,999998,999998,999998,999998,999998,999998,999998,999998,999998


In [5]:
df.shape[0]*df.shape[1]

25000000

In [14]:
def f1(df):
    return df.shape[0]*df.shape[1]

f1(df)

25000000

In [13]:
f2 = lambda d: d.shape[0]*d.shape[1]
f2(df)

25000000

## **1. Benchmarks**
- **컬럼 or 행**

In [3]:
take_cols=['a', 'b', 'g', 'n', 'x']

query = "a < 50_000 and b > 3000"

In [ ]:
%%timeit -n 100
subdf = df[take_cols]
subdf = subdf[subdf['a'] < 50_000]
subdf = subdf[subdf['b'] > 3000]
subdf

22.4 ms ± 926 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


In [ ]:
%%timeit -n 100
subdf = df[df['a'] < 50_000]
subdf = subdf[subdf['b'] > 3000]
subdf = subdf[take_cols]
subdf

10.4 ms ± 818 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


In [ ]:
%%timeit -n 100
# first take columns then filter rows
df.filter(take_cols).query(query)

27.4 ms ± 1.56 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit -n 100
# first filter rows then take columns
df.query(query).filter(take_cols)

14.7 ms ± 1.69 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
n_of_elements = lambda d: d.shape[0]*d.shape[1]

In [ ]:
n_of_elements(df)

25000000

In [ ]:
n_of_elements(df.filter(take_cols))

5000000

In [ ]:
25000000 > 5000000

True

In [ ]:
n_of_elements(df.query(query))

1174975

In [ ]:
5000000 > 1174975

True

### <font color='blue'>**Bracketing: filter rows, then select columns (10.7 ms)**

## **2. Benchmarks**
### **More columns than rows**

In [ ]:
n = 1_000_000
df = pd.DataFrame({
    f'x{i}': list(range(25))
    for i in range(1_000_000)
})
df

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x999990,x999991,x999992,x999993,x999994,x999995,x999996,x999997,x999998,x999999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
5,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
6,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
7,7,7,7,7,7,7,7,7,7,7,...,7,7,7,7,7,7,7,7,7,7
8,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8
9,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9


In [ ]:
n_of_elements = lambda d: d.shape[0]*d.shape[1]
n_of_elements(df)

25000000

In [ ]:
take_cols = ['x1', 'x2', 'x78', 'x3456', 'x9999']

In [ ]:
%%timeit -n 100
subdf = df[take_cols]
subdf = subdf[subdf['x1'] < 20]
subdf = subdf[subdf['x2'] > 5]
subdf

1.45 ms ± 41.4 µs per loop (mean ± std. dev. of 7 runs, 500 loops each)


In [ ]:
%%timeit -n 100
subdf = df[df['x1'] < 20]
subdf = subdf[subdf['x2'] > 5]
subdf = subdf[take_cols]
subdf

108 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
n_of_elements(df[take_cols])

125

In [ ]:
n_of_elements(df[df['x1'] < 20])

20000000

In [ ]:
subdf = df[take_cols]
subdf = subdf[subdf['x1'] < 20]

In [ ]:
n_of_elements(subdf)

100

In [ ]:
subdf = df[df['x1'] < 20]
subdf = subdf[subdf['x2'] > 5]

In [ ]:
n_of_elements(subdf)

14000000

### **Reducing the dataset size as soon as possible, particularly through vectorized row filtering operations, can significantly improve performance.**

## **END**

In [15]:
n = 1_000_000
df1 = pd.DataFrame({
    letter: list(range(n))
    for letter in "abcde"
})
df1

,a,b,c,d,e
0,0,0,0,0,0
1,1,1,1,1,1
2,2,2,2,2,2
3,3,3,3,3,3
4,4,4,4,4,4
...,...,...,...,...,...
999995,999995,999995,999995,999995,999995
999996,999996,999996,999996,999996,999996
999997,999997,999997,999997,999997,999997
999998,999998,999998,999998,999998,999998


In [23]:
n = 1_000_000
df2 = pd.DataFrame({
    letter: list(range(n*3))
    for letter in "defgh"
})
df2

,d,e,f,g,h
0,0,0,0,0,0
1,1,1,1,1,1
2,2,2,2,2,2
3,3,3,3,3,3
4,4,4,4,4,4
...,...,...,...,...,...
2999995,2999995,2999995,2999995,2999995,2999995
2999996,2999996,2999996,2999996,2999996,2999996
2999997,2999997,2999997,2999997,2999997,2999997
2999998,2999998,2999998,2999998,2999998,2999998


In [25]:
pd.merge(df1, df2)

,a,b,c,d,e,f,g,h
0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1
2,2,2,2,2,2,2,2,2
3,3,3,3,3,3,3,3,3
4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...
999995,999995,999995,999995,999995,999995,999995,999995,999995
999996,999996,999996,999996,999996,999996,999996,999996,999996
999997,999997,999997,999997,999997,999997,999997,999997,999997
999998,999998,999998,999998,999998,999998,999998,999998,999998


In [26]:
pd.merge(df1, df2, how='inner')

,a,b,c,d,e,f,g,h
0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1
2,2,2,2,2,2,2,2,2
3,3,3,3,3,3,3,3,3
4,4,4,4,4,4,4,4,4
...,...,...,...,...,...,...,...,...
999995,999995,999995,999995,999995,999995,999995,999995,999995
999996,999996,999996,999996,999996,999996,999996,999996,999996
999997,999997,999997,999997,999997,999997,999997,999997,999997
999998,999998,999998,999998,999998,999998,999998,999998,999998


In [27]:
pd.merge(df1, df2, how='outer')

,a,b,c,d,e,f,g,h
0,0.0,0.0,0.0,0,0,0,0,0
1,1.0,1.0,1.0,1,1,1,1,1
2,2.0,2.0,2.0,2,2,2,2,2
3,3.0,3.0,3.0,3,3,3,3,3
4,4.0,4.0,4.0,4,4,4,4,4
...,...,...,...,...,...,...,...,...
2999995,NaN,NaN,NaN,2999995,2999995,2999995,2999995,2999995
2999996,NaN,NaN,NaN,2999996,2999996,2999996,2999996,2999996
2999997,NaN,NaN,NaN,2999997,2999997,2999997,2999997,2999997
2999998,NaN,NaN,NaN,2999998,2999998,2999998,2999998,2999998


# **Pandas Study**
## **[PythonDataScienceHandbook](https://jakevdp.github.io/PythonDataScienceHandbook/)**
### **[Combining Datasets: Merge and Join](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html)**